In [2]:
from google.transit import gtfs_realtime_pb2
import urllib
import time
import datetime

from bs4 import BeautifulSoup
import requests
import re

In [31]:
## pulling info from api

feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.urlopen('http://datamine.mta.info/mta_esi.php?key=5a8f6f4375e9f4bdea9b0c86afeaf911')
feed.ParseFromString(response.read())
for entity in feed.entity:
  if entity.HasField('trip_update'):
    print entity.trip_update


trip {
  trip_id: "102300_1..N02R"
  start_date: "20150722"
  route_id: "1"
}
stop_time_update {
  arrival {
    time: 1437602860
  }
  stop_id: "101N"
}

trip {
  trip_id: "102600_1..S02R"
  start_date: "20150722"
  route_id: "1"
}
stop_time_update {
  arrival {
    time: 1437602866
  }
  stop_id: "140S"
}

trip {
  trip_id: "102700_1..N02R"
  start_date: "20150722"
  route_id: "1"
}
stop_time_update {
  arrival {
    time: 1437602879
  }
  departure {
    time: 1437602909
  }
  stop_id: "103N"
}
stop_time_update {
  arrival {
    time: 1437602999
  }
  stop_id: "101N"
}

trip {
  trip_id: "103000_1..S02R"
  start_date: "20150722"
  route_id: "1"
}
stop_time_update {
  arrival {
    time: 1437602803
  }
  departure {
    time: 1437602923
  }
  stop_id: "139S"
}
stop_time_update {
  arrival {
    time: 1437603073
  }
  stop_id: "140S"
}

trip {
  trip_id: "103100_1..N02R"
  start_date: "20150722"
  route_id: "1"
}
stop_time_update {
  arrival {
    time: 1437602898
  }
  departure {
  

In [24]:
len(feed.entity)

434

In [25]:
# trip ID
feed.entity[2].trip_update.trip.trip_id

u'102600_1..S02R'

In [26]:
# length of stop time update
len(feed.entity[2].trip_update.stop_time_update)

1

In [27]:
# stop ID
feed.entity[2].trip_update.stop_time_update[1].stop_id

IndexError: list index out of range

In [28]:
# time
feed.entity[2].trip_update.stop_time_update[1].arrival.time

IndexError: list index out of range

In [29]:
#system time
datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')

'2015-07-22 18:07:18'

In [30]:
print feed.entity[2]

id: "000003"
trip_update {
  trip {
    trip_id: "102600_1..S02R"
    start_date: "20150722"
    route_id: "1"
  }
  stop_time_update {
    arrival {
      time: 1437602866
    }
    stop_id: "140S"
  }
}



In [10]:
### save in file definition
def savefile(listitems):
    # 'a' to add line to txt file
    with open('mta.txt', 'a') as f:
            f.write(listitems + '\n')


In [11]:
## draft of the loop

#system time
tmpSys = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')

i = 1

entity=feed.entity[2]
if entity.HasField('trip_update'):
    
    ## trip update: 3 data characterisitcs
    TripIdi = entity.trip_update.trip.trip_id
    StartDatei = entity.trip_update.trip.start_date
    Routei = entity.trip_update.trip.route_id
    
    ## trip alert: 1 data characterisitcs
    if entity.HasField('alert'):
        Alerti = entity.alert.header_text
    else:
        Alerti = ''
    
    ## looking to record first 3 predicted arrival time 
    Arrivetmp=[]
    stop_id=[]
    
    tripcomplete = []
    
    if len(entity.trip_update.stop_time_update) <> 0:
        #record only first three trip updates
        p3trip = min(len(entity.trip_update.stop_time_update),3)
        
        j=0
        while j < p3trip:
            tripcomplete = []
            #print j
            ArrivalTime = entity.trip_update.stop_time_update[j].arrival.time
            stop_id = entity.trip_update.stop_time_update[j].stop_id
            
            tripcomplete.extend([tmpSys,TripIdi,StartDatei,Routei,Alerti,stop_id,ArrivalTime])
            print tripcomplete, type(tripcomplete)
            tripcomplstr = ','.join([str(i) for i in tripcomplete])
            savefile(tripcomplstr)
            
            j+=1

    


['2015-07-07 22:30:47', u'128800_1..N02R', u'20150707', u'1', '', u'101N', 1436322539L] <type 'list'>


In [12]:
# Perturbation on MTA info -- webscraping of mta.info :

#definition of extracting function 

def extract_url_content(urlv, headersv):
        # Request by url and headers
        req =  requests.get(urlv, headers=headers)
        demande = req.text
        # Request status code
        statut_demande = req.status_code
        if statut_demande/100 in [4,5]:
            return 'error on requests with error: ', statut_demande
        return BeautifulSoup(demande)

In [13]:
# Perturbation on MTA info -- webscraping of mta.info :

#draft for the delays for different lines from mta.info

url = 'http://www.mta.info/status/subway/123'

headers = None

texturl = extract_url_content(url, headers)
print texturl.prettify

<bound method BeautifulSoup.prettify of <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="http://www.mta.info/sites/default/files/apple-touch-icon-precomposed.png" rel="apple-touch-icon-precomposed" type="image/png"/>
<link href="http://www.mta.info/sites/default/files/apple-touch-icon.png" rel="apple-touch-icon" type="image/png

In [14]:
# Perturbation on MTA info -- webscraping of mta.info :

## draft of extraction of lines perturbated

Delaystext = texturl.find(class_="TitleDelay").get_text()
print Delaystext

lineaffected001 = texturl.findAll(id='status_display')
#print lineaffected001
lineaffected002 = [i.findAll('img') for i in lineaffected001]
print lineaffected002
print len(lineaffected002)
for j in range(len(lineaffected002)):
    textaffected001 = [i['alt'][:1] for i in lineaffected002[j]]
textaffected001

AttributeError: 'NoneType' object has no attribute 'get_text'

In [15]:

## perturbations on MTA lines are publised here:

urllist = [
    'http://www.mta.info/status/subway/123',
    'http://www.mta.info/status/subway/456',
    'http://www.mta.info/status/subway/7',
    'http://www.mta.info/status/subway/ACE',
    'http://www.mta.info/status/subway/BDFM',
    'http://www.mta.info/status/subway/JZ',
    'http://www.mta.info/status/subway/L',
    'http://www.mta.info/status/subway/NQR',
    'http://www.mta.info/status/subway/S'
]


In [16]:
## Summary of the loop
def listofperturbations(listOfLines):
    
    ## webscraping of mta/info using the list of url listed above
    linesperturbated = []
    for url in listOfLines:
        headers = None
        urlbody = extract_url_content(url, headers)
        #print url, 'find TitleDelay',  urlbody.find(class_="TitleDelay")
        
        if urlbody.find(class_="TitleDelay") <> None:
            Delaystext = urlbody.find(class_="TitleDelay").get_text()
            #print 'delaystext', Delaystext
            if Delaystext == "Delays":
                lineaffected001 = urlbody.findAll(id='status_display')
                lineaffected002 = [i.findAll('img') for i in lineaffected001]
                #print 'line affectd 002', lineaffected002
            for j in range(len(lineaffected002)):
                    #print j
                    for i in lineaffected002[j]:
                        try:
                            linesperturbated.extend(i['alt'][:1])
                        except:
                            continue
                            
                    #textaffected001 = [i for i in lineaffected002[j]]
                    #print textaffected001
                    #linesperturbated.extend(textaffected001)
    return linesperturbated 

In [17]:
listofperturbations(urllist)

[]

In [19]:
def loopdwlmta(x):
    
    #system time for reference
    tmpSys = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
    
    ##load list of lines perturbated
    listlineperturbated = listofperturbations(urllist)

    ## send request to api mta.info using my own key
    feed = gtfs_realtime_pb2.FeedMessage()
    response = urllib.urlopen('http://datamine.mta.info/mta_esi.php?key=5a8f6f4375e9f4bdea9b0c86afeaf911')
    feed.ParseFromString(response.read())
    
    
    ## looping the content downloaded
    for entity in feed.entity:
        if entity.HasField('trip_update'):
    
            ## trip update: 3 data characterisitcs
            TripIdi = entity.trip_update.trip.trip_id
            StartDatei = entity.trip_update.trip.start_date
            Routei = entity.trip_update.trip.route_id
            if Routei in listlineperturbated:
                Delaysi = "delays"
            else:
                Delaysi = ""

            ## trip alert: 1 data characterisitcs
            if entity.HasField('alert'):
                Alerti = entity.alert.header_text
            else:
                Alerti = ''

            ## looking to record first 3 predicted arrival time 
            Arrivetmp=[]
            stop_id=[]

            tripcomplete = []

            if len(entity.trip_update.stop_time_update) <> 0:
                #record only first three trip updates
                p3trip = min(len(entity.trip_update.stop_time_update),1)

                j=0
                while j < p3trip:
                    tripcomplete = []
                    #print j
                    ArrivalTime = entity.trip_update.stop_time_update[j].arrival.time
                    stop_id = entity.trip_update.stop_time_update[j].stop_id
                    station_nth=j

                    tripcomplete.extend([tmpSys,TripIdi,StartDatei,Routei,Alerti,station_nth,stop_id,ArrivalTime,Delaysi])
                    #print tripcomplete, type(tripcomplete)
                    tripcomplstr = ','.join([str(i) for i in tripcomplete])
                    savefile(tripcomplstr)

                    j+=1

In [20]:
# test function defined above
loopdwlmta(1)

In [22]:
linesperturbated = []
for url in urllist:
    headers = None
    texturl2 = extract_url_content(url, headers)
    print  texturl2.find(class_="TitleDelay") 
    if texturl2.find(class_="TitleDelay") <> None:
        Delaystext = texturl2.find(class_="TitleDelay").get_text()
        if Delaystext == "Delays":
            lineaffected001 = texturl2.findAll(id='status_display')
            lineaffected002 = [i.findAll('img') for i in lineaffected001]
            print lineaffected002
            for j in range(len(lineaffected002)):
                    #print j
                    for i in lineaffected002[j]:
                        try:
                            linesperturbated.extend(i['alt'][:1])
                        except:
                            continue
                            
                    #textaffected001 = [i for i in lineaffected002[j]]
                    #print textaffected001
                    #linesperturbated.extend(textaffected001)


None
None
None
None
None
None
None
None
None


In [23]:
linesperturbated

[]

In [ ]:
import time 
while True:
    try: 
        loopdwlmta(1)
    except:
        continue
    time.sleep(300)